## 2 Methods (as in point 2)

1. Use model trained on
    - current 2001, prediction_2003-2009, FEMP_2001, LIT_2001 to give output on 2011

2. Give it input as
    - current_2011, prediction_2011-2017 (obtained using model trained on 2011), FEMP_2011, LIT_2011 to give output on 2019
    - current_2011, prediction_2013-2019 (obtained using model trained on 2011), FEMP_2011, LIT_2011 to give output on 2019

In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, accuracy_score

Using TensorFlow backend.
/Users/arpitjain/anaconda3/envs/automl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/arpitjain/anaconda3/envs/automl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/arpitjain/anaconda3/envs/automl_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

In [ ]:
# BC- backward classifier jisme 2011 ke model se 2009,07,05,03,01 predicted+2011census+EMP2011+LIT2011 
# Ye sab use krke 2001 ke labels predict kiye the.

# FC- forward classifier
# Isme 2001 wale model se 2011 predict kia tha. Using census2001+intermediate years+ emp+lit

# Isi tarah humne forward classifier 2011 pr train krke 2019 predict kia tha

In [53]:
# tss = pickle.load( open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/Temporal_Transferability/CHH_2011_TT_FEMP&LIT_base_XGB.pkl", 'rb'))
tss = pickle.load( open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/Temporal_Transferability/FORMAL_EMP_2011_TT_Current+base_XGB.pkl", 'rb'))
print('val_f1_score ',tss['xgBoost']['val_scores'])
print('train_f1_score ',tss['xgBoost']['train_scores'])

val_f1_score  [0.8830784632692292, 0.8797370335434369, 0.8700843467438716, 0.8696449091416723, 0.8675950051710682]
train_f1_score  [0.9101612730413775, 0.9117291760481253, 0.9108298530327632, 0.9128374577077851, 0.9134412114023387]


In [54]:
n_estimators = tss['xgBoost']['specs'][0]['n_estimators']
max_depth = tss['xgBoost']['specs'][0]['max_depth']
learning_rate = tss['xgBoost']['specs'][0]['learning_rate']
objective = tss['xgBoost']['specs'][0]['objective']
booster = tss['xgBoost']['specs'][0]['booster']
gamma = tss['xgBoost']['specs'][0]['gamma']
min_child_weight = tss['xgBoost']['specs'][0]['min_child_weight']
max_delta_step = tss['xgBoost']['specs'][0]['max_delta_step']
subsample = tss['xgBoost']['specs'][0]['subsample']
colsample_bytree = tss['xgBoost']['specs'][0]['colsample_bytree']
colsample_bylevel = tss['xgBoost']['specs'][0]['colsample_bylevel']
colsample_bynode = tss['xgBoost']['specs'][0]['colsample_bynode']
reg_alpha = tss['xgBoost']['specs'][0]['reg_alpha']
reg_lambda = tss['xgBoost']['specs'][0]['reg_lambda']
scale_pos_weight = tss['xgBoost']['specs'][0]['scale_pos_weight']
base_score = tss['xgBoost']['specs'][0]['base_score']

n_splits = tss['xgBoost']['specs'][0]['kFold_splits']

print('                 ')
print('Re-Training again for calculating f1 and train scores')

xgbc = XGBClassifier(n_estimators=n_estimators, 
         max_depth=max_depth, learning_rate=learning_rate, 
         objective=objective, booster=booster,n_jobs=-1, 
         gamma=gamma, min_child_weight=min_child_weight, 
         max_delta_step=max_delta_step, subsample=subsample, 
         colsample_bytree=colsample_bytree, 
         colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
         reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
         scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

cv = KFold(n_splits=n_splits, shuffle=True)

                 
Re-Training again for calculating f1 and train scores


In [55]:
# For all except LIT and FEMP
ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")
cols_2 = ['census_code', 'LIT_2001', 'FEMP_2001']
femp_lit = femp_lit[cols_2]

indicator = 'CHH'

cols = ['census_code', indicator+'_2001', indicator+'_2011']
gt = ground_truth[cols]

df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data/CHH_CC.csv")

data = gt.merge(df,on='census_code', how='left')
data = femp_lit.merge(data, on='census_code', how='left')
data.dropna(inplace=True)

In [14]:
data.head()

,census_code,LIT_2001,FEMP_2001,CHH_2001,CHH_2011,predictions_2001,predictions_2003,predictions_2005,predictions_2007,predictions_2009,predictions_2011
0,1,1,2,1,1,1.0,3.0,1.0,1.0,1.0,1.0
1,2,1,2,1,2,1.0,1.0,1.0,1.0,1.0,1.0
3,4,1,3,1,1,1.0,2.0,2.0,2.0,2.0,2.0
4,5,1,2,1,1,1.0,3.0,3.0,3.0,3.0,3.0
5,6,1,2,1,2,1.0,3.0,1.0,3.0,3.0,3.0


In [56]:
# For LIT and FEMP

# ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")
# cols_2 = ['census_code', 'LIT_2001', 'FEMP_2001']
# femp_lit = femp_lit[cols_2]

# For LIT
# indicator = 'LIT'
# cols = ['census_code', indicator+'_2001', indicator+'_2011', 'FEMP_2001']

# For FEMP
indicator = 'FEMP'
cols = ['census_code', indicator+'_2001', indicator+'_2011', 'LIT_2001']

gt = femp_lit[cols]

# df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data/CHH_CC.csv")
df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data/FEMP_CC.csv")

data = gt.merge(df,on='census_code', how='left')
# data = femp_lit.merge(data, on='census_code', how='left')
data.dropna(inplace=True)

In [57]:
data.head(5)

,census_code,FEMP_2001,FEMP_2011,LIT_2001,predictions_2001,predictions_2003,predictions_2005,predictions_2007,predictions_2009,predictions_2011
0,1,2,2,1,2.0,2.0,2.0,2.0,3.0,2.0
1,2,2,2,1,2.0,2.0,2.0,2.0,3.0,2.0
3,4,3,3,1,3.0,3.0,3.0,3.0,3.0,3.0
4,5,2,2,1,2.0,2.0,2.0,2.0,2.0,2.0
5,6,2,2,1,2.0,2.0,2.0,2.0,2.0,2.0


In [58]:
# feature_cols = [indicator+'_2001','LIT_2001','FEMP_2001','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

# For Lit
# feature_cols = [indicator+'_2001','FEMP_2001','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

# For FEMP
feature_cols = [indicator+'_2001','LIT_2001','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

target = indicator+'_2011'

In [59]:
X = data[feature_cols].values
y = data[target].values

In [60]:
census_codes = data['census_code'].values

In [61]:
val_f1score = []
val_accscore = []
train_f1score = []
train_accscore = []
# f1score = []
# accscore = []
final_df_fc = []
counter=0
for train_index, test_index in cv.split(X):
    
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    
    best_xgb = xgbc.fit(X_train, y_train)
    
#     prediction_2011 = best_xgb.predict(X_2011)
#     f1_weight = f1_score(y_2011, prediction_2011, average='weighted')
#     acc = accuracy_score(y_2011, prediction_2011)

#     f1score.append(f1_weight)
#     accscore.append(acc)
    
    census_split = census_codes[test_index]
    predictions = best_xgb.predict(X_test)
    predictions_train = best_xgb.predict(X_train)
    
    final_predictions = np.stack((census_split, predictions))
    final_predictions_df = pd.DataFrame(final_predictions)
    final_predictions_df = final_predictions_df.transpose()
    final_predictions_df.columns = ['census_code', 'CHH_prediction_2011']
    final_df_fc.append(final_predictions_df)
    

    f1_weight = f1_score(y_test, predictions, average='weighted')
    f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
    acc = accuracy_score(y_test, predictions)
    acc_train = accuracy_score(y_train, predictions_train)
    print(counter, f1_weight, f1_weight_train, acc, acc_train)
    counter=counter+1

    val_f1score.append(f1_weight)
    train_f1score.append(f1_weight_train)
    val_accscore.append(acc)
    train_accscore.append(acc_train)

val_f1score = np.array(val_f1score).mean()
train_f1score = np.array(train_f1score).mean()
val_accscore = np.array(val_accscore).mean()
train_accscore = np.array(train_accscore).mean()
print('------')
print(val_f1score, train_f1score, val_accscore, train_accscore)

0 0.8892221190341597 0.9292642840692587 0.8879310344827587 0.9292364990689013
1 0.8271220159151195 0.9260971987983836 0.8275862068965517 0.9259259259259259
2 0.8954632715155028 0.9282973502772535 0.896551724137931 0.9281767955801105
3 0.8606859172039996 0.93373757466658 0.8620689655172413 0.9333333333333333
4 0.8645160908617594 0.9415772628977244 0.8620689655172413 0.9413919413919414
------
0.8674018829061081 0.93179473414184 0.8672413793103448 0.9316128990600425


In [62]:
dd = pd.concat(final_df_fc)

In [63]:
dd.to_csv('Results_Chahat/FEMP_FC_2011.csv',index=False)

In [9]:
# Model has been trained now

In [2]:
tss = pickle.load( open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/Temporal_Transferability/FC_2001_TT_FEMP&LIT_2011_base_XGB.pkl", 'rb'))
# tss = pickle.load( open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/Temporal_Transferability/LIT_2001_TT_Current+base_XGB.pkl", 'rb'))
print('val_f1_score ',tss['xgBoost']['val_scores'])
print('train_f1_score ',tss['xgBoost']['train_scores'])

val_f1_score  [0.8790338989833988, 0.8730276008430247, 0.8687923097390229, 0.8643965745654454, 0.8562879355679154]
train_f1_score  [0.9474524837233117, 0.949991423692485, 0.95726378143306, 0.9581513592364045, 0.9559358390455802]


In [3]:
n_estimators = tss['xgBoost']['specs'][0]['n_estimators']
max_depth = tss['xgBoost']['specs'][0]['max_depth']
learning_rate = tss['xgBoost']['specs'][0]['learning_rate']
objective = tss['xgBoost']['specs'][0]['objective']
booster = tss['xgBoost']['specs'][0]['booster']
gamma = tss['xgBoost']['specs'][0]['gamma']
min_child_weight = tss['xgBoost']['specs'][0]['min_child_weight']
max_delta_step = tss['xgBoost']['specs'][0]['max_delta_step']
subsample = tss['xgBoost']['specs'][0]['subsample']
colsample_bytree = tss['xgBoost']['specs'][0]['colsample_bytree']
colsample_bylevel = tss['xgBoost']['specs'][0]['colsample_bylevel']
colsample_bynode = tss['xgBoost']['specs'][0]['colsample_bynode']
reg_alpha = tss['xgBoost']['specs'][0]['reg_alpha']
reg_lambda = tss['xgBoost']['specs'][0]['reg_lambda']
scale_pos_weight = tss['xgBoost']['specs'][0]['scale_pos_weight']
base_score = tss['xgBoost']['specs'][0]['base_score']

n_splits = tss['xgBoost']['specs'][0]['kFold_splits']

print('                 ')
print('Re-Training again for calculating f1 and train scores')

xgbc = XGBClassifier(n_estimators=n_estimators, 
         max_depth=max_depth, learning_rate=learning_rate, 
         objective=objective, booster=booster,n_jobs=-1, 
         gamma=gamma, min_child_weight=min_child_weight, 
         max_delta_step=max_delta_step, subsample=subsample, 
         colsample_bytree=colsample_bytree, 
         colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
         reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
         scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

cv = KFold(n_splits=n_splits, shuffle=True)

                 
Re-Training again for calculating f1 and train scores


In [4]:
tss['xgBoost']['specs'][0]

{'n_estimators': 1000,
 'max_depth': 3,
 'learning_rate': 0.1,
 'verbosity': 1,
 'objective': 'multi:softmax',
 'booster': 'gbtree',
 'n_jobs': -1,
 'gamma': 0,
 'min_child_weight': 8,
 'max_delta_step': 6,
 'subsample': 0.7,
 'colsample_bytree': 1,
 'colsample_bylevel': 1,
 'colsample_bynode': 0.9,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'base_score': 0.5,
 'random_state': 0,
 'importance_type': 'gain',
 'n_randomized_search': 200,
 'run_type': 'train-test',
 'trail': 0,
 'trail_type': 'sales',
 'train_ratio': 0.9,
 'val_train_loss': 'f1_weighted',
 'top_n_models_drill': 5,
 'top_n_models_pred': 5,
 'n_drill_search': [100],
 'test_ratio': 0.0,
 'y_col': 'FC_2001',
 'cols_to_drop': [],
 'ensemble': False,
 'n_cols_dropped': 0,
 'current_date_col': 'census_code',
 'no_steps_ahead': 0,
 'apply_smote': True,
 'time_cols': [],
 'cross_validation': ['KFold'],
 'kFold_splits': 5,
 'pFold_splits': 20,
 'feature_cols': ['FC_2011',
  'LIT_2011',
  'FEMP_2011',
  'predictions

In [5]:
# For All except LIT and FEMP

ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")
cols_2 = ['census_code', 'LIT_2011', 'FEMP_2011']
femp_lit = femp_lit[cols_2]

indicator = 'FC'

cols = ['census_code', indicator+'_2001', indicator+'_2011']
gt = ground_truth[cols]

df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data_2011/FC_CC.csv")

data = gt.merge(df,on='census_code', how='left')
data = femp_lit.merge(data, on='census_code', how='left')
data.dropna(inplace=True)

In [6]:
data.head(5)

,census_code,LIT_2011,FEMP_2011,FC_2001,FC_2011,predictions_2001,predictions_2003,predictions_2005,predictions_2007,predictions_2009,predictions_2011
0,1,1,2,1,1,1.0,1.0,1.0,1.0,1.0,1.0
1,2,1,2,2,1,1.0,1.0,1.0,1.0,1.0,1.0
3,4,2,3,2,1,3.0,1.0,1.0,1.0,1.0,1.0
4,5,1,2,1,1,1.0,1.0,1.0,1.0,1.0,1.0
5,6,1,2,1,1,1.0,1.0,1.0,1.0,1.0,1.0


In [21]:
# For LIT and FEMP

femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")

# For LIT
indicator = 'LIT'
cols = ['census_code', indicator+'_2001', indicator+'_2011', 'FEMP_2011']

# For FEMP
# indicator = 'FEMP'
# cols = ['census_code', indicator+'_2001', indicator+'_2011', 'LIT_2011']

gt = femp_lit[cols]

# df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data/CHH_CC.csv")
df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data_2011/LIT_CC.csv")

data = gt.merge(df,on='census_code', how='left')
# data = femp_lit.merge(data, on='census_code', how='left')
data.dropna(inplace=True)

In [22]:
data.head()

,census_code,LIT_2001,LIT_2011,FEMP_2011,predictions_2001,predictions_2003,predictions_2005,predictions_2007,predictions_2009,predictions_2011
0,1,1,1,2,1.0,3.0,1.0,1.0,1.0,1.0
1,2,1,1,2,1.0,1.0,1.0,1.0,1.0,1.0
3,4,1,2,3,3.0,2.0,2.0,2.0,2.0,2.0
4,5,1,1,2,1.0,3.0,1.0,3.0,1.0,1.0
5,6,1,1,2,1.0,3.0,1.0,1.0,1.0,1.0


In [24]:
# feature_cols = [indicator+'_2011','LIT_2011','FEMP_2011','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

# feature_cols = [indicator+'_2001','LIT_2001','FEMP_2001','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

# For Lit
feature_cols = [indicator+'_2011','FEMP_2011','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

# For FEMP
# feature_cols = [indicator+'_2011','LIT_2011','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

target = indicator+'_2001'

In [25]:
data[feature_cols].head(2)

,LIT_2011,FEMP_2011,predictions_2003,predictions_2005,predictions_2007,predictions_2009
0,1,2,3.0,1.0,1.0,1.0
1,1,2,1.0,1.0,1.0,1.0


In [26]:
data[target].head(2)

0    1
1    1
Name: LIT_2001, dtype: int64

In [27]:
X = data[feature_cols].values
y = data[target].values

In [28]:
census_codes = data['census_code'].values

In [29]:
val_f1score = []
val_accscore = []
train_f1score = []
train_accscore = []
# f1score = []
# accscore = []
final_df_fc = []
counter=0
for train_index, test_index in cv.split(X):
    
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    
    best_xgb = xgbc.fit(X_train, y_train)
    
#     prediction_2011 = best_xgb.predict(X_2011)
#     f1_weight = f1_score(y_2011, prediction_2011, average='weighted')
#     acc = accuracy_score(y_2011, prediction_2011)

#     f1score.append(f1_weight)
#     accscore.append(acc)
    
    census_split = census_codes[test_index]
    predictions = best_xgb.predict(X_test)
    predictions_train = best_xgb.predict(X_train)
    
    final_predictions = np.stack((census_split, predictions))
    final_predictions_df = pd.DataFrame(final_predictions)
    final_predictions_df = final_predictions_df.transpose()
    final_predictions_df.columns = ['census_code', 'LIT_prediction_2001']
    final_df_fc.append(final_predictions_df)
    

    f1_weight = f1_score(y_test, predictions, average='weighted')
    f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
    acc = accuracy_score(y_test, predictions)
    acc_train = accuracy_score(y_train, predictions_train)
    print(counter, f1_weight, f1_weight_train, acc, acc_train)
    counter=counter+1

    val_f1score.append(f1_weight)
    train_f1score.append(f1_weight_train)
    val_accscore.append(acc)
    train_accscore.append(acc_train)

val_f1score = np.array(val_f1score).mean()
train_f1score = np.array(train_f1score).mean()
val_accscore = np.array(val_accscore).mean()
train_accscore = np.array(train_accscore).mean()
print('------')
print(val_f1score, train_f1score, val_accscore, train_accscore)

0 0.7575757575757576 0.8640450063108482 0.7758620689655172 0.867595818815331
1 0.8081819865120813 0.8520874986967155 0.8103448275862069 0.8588807785888077
2 0.7733427163969037 0.869639969356282 0.7844827586206896 0.8736842105263158
3 0.7610841916703985 0.8670985455723776 0.7672413793103449 0.8711864406779661
4 0.6982276247967143 0.8676708507429746 0.7068965517241379 0.8722415795586528
------
0.759682455390371 0.8641083741358395 0.7689655172413793 0.8687177656334146


In [30]:
dd = pd.concat(final_df_fc)

In [31]:
dd.to_csv('LIT_BC_2001.csv',index=False)

1. Use model trained on
    - current 2001, prediction_2003-2009, FEMP_2001, LIT_2001 to give output on 2011

2. Give it input as
    - current_2011, prediction_2011-2017 (obtained using model trained on 2011), FEMP_2011, LIT_2011 to give output on 2019
    - current_2011, prediction_2013-2019 (obtained using model trained on 2011), FEMP_2011, LIT_2011 to give output on 2019

In [98]:
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, accuracy_score

In [382]:
# tss = pickle.load( open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/Temporal_Transferability/MSW_2011_TT_FEMP&LIT_base_XGB.pkl", 'rb'))
# tss = pickle.load( open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/Temporal_Transferability/MSW_2001_TT_FEMP&LIT_2011_base_XGB.pkl", 'rb'))
tss = pickle.load( open("/Users/arpitjain/Downloads/SatPRo/mlpros/classifier_cv/Temporal_Transferability/FORMAL_EMP_2001_TT_Current+base_XGB.pkl", 'rb'))
print('val_f1_score ',tss['xgBoost']['val_scores'])
print('train_f1_score ',tss['xgBoost']['train_scores'])

val_f1_score  [0.8913233525244925, 0.8811188053001414, 0.8808806812808957, 0.8796172436073361, 0.872538909198773]
train_f1_score  [0.9157949654427109, 0.9149759958846001, 0.9305906802828747, 0.9276685157918857, 0.9319774091533223]


In [383]:
n_estimators = tss['xgBoost']['specs'][0]['n_estimators']
max_depth = tss['xgBoost']['specs'][0]['max_depth']
learning_rate = tss['xgBoost']['specs'][0]['learning_rate']
objective = tss['xgBoost']['specs'][0]['objective']
booster = tss['xgBoost']['specs'][0]['booster']
gamma = tss['xgBoost']['specs'][0]['gamma']
min_child_weight = tss['xgBoost']['specs'][0]['min_child_weight']
max_delta_step = tss['xgBoost']['specs'][0]['max_delta_step']
subsample = tss['xgBoost']['specs'][0]['subsample']
colsample_bytree = tss['xgBoost']['specs'][0]['colsample_bytree']
colsample_bylevel = tss['xgBoost']['specs'][0]['colsample_bylevel']
colsample_bynode = tss['xgBoost']['specs'][0]['colsample_bynode']
reg_alpha = tss['xgBoost']['specs'][0]['reg_alpha']
reg_lambda = tss['xgBoost']['specs'][0]['reg_lambda']
scale_pos_weight = tss['xgBoost']['specs'][0]['scale_pos_weight']
base_score = tss['xgBoost']['specs'][0]['base_score']

n_splits = tss['xgBoost']['specs'][0]['kFold_splits']

print('                 ')
print('Re-Training again for calculating f1 and train scores')

xgbc = XGBClassifier(n_estimators=n_estimators, 
         max_depth=max_depth, learning_rate=learning_rate, 
         objective=objective, booster=booster,n_jobs=-1, 
         gamma=gamma, min_child_weight=min_child_weight, 
         max_delta_step=max_delta_step, subsample=subsample, 
         colsample_bytree=colsample_bytree, 
         colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
         reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
         scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

cv = KFold(n_splits=n_splits, shuffle=True)

                 
Re-Training again for calculating f1 and train scores


In [340]:
# For All except Lit and FEMP
ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")
cols_2 = ['census_code', 'LIT_2001', 'FEMP_2001']
femp_lit = femp_lit[cols_2]

indicator = 'MSW'

cols = ['census_code', indicator+'_2001', indicator+'_2011']
gt = ground_truth[cols]

df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data_2011/MSW_CC.csv")

data = gt.merge(df,on='census_code', how='left')
data = femp_lit.merge(data, on='census_code', how='left')
data.dropna(inplace=True)

In [384]:
# For LIT and FEMP

femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")

# For LIT
# indicator = 'LIT'
# cols = ['census_code', indicator+'_2001', indicator+'_2011', 'FEMP_2011']

# For FEMP
indicator = 'FEMP'
cols = ['census_code', indicator+'_2001', indicator+'_2011', 'LIT_2011']

gt = femp_lit[cols]

# df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data/CHH_CC.csv")
df = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data_2011/FEMP_CC.csv")

data = gt.merge(df,on='census_code', how='left')
# data = femp_lit.merge(data, on='census_code', how='left')
data.dropna(inplace=True)

In [385]:
data.head(5)

,census_code,FEMP_2001,FEMP_2011,LIT_2011,predictions_2001,predictions_2003,predictions_2005,predictions_2007,predictions_2009,predictions_2011
0,1,2,2,1,2.0,3.0,2.0,2.0,2.0,2.0
1,2,2,2,1,3.0,2.0,2.0,2.0,2.0,2.0
3,4,3,3,2,2.0,3.0,3.0,3.0,3.0,3.0
4,5,2,2,1,3.0,3.0,3.0,3.0,2.0,2.0
5,6,2,2,1,2.0,3.0,2.0,2.0,2.0,2.0


In [386]:
data.head()

,census_code,FEMP_2001,FEMP_2011,LIT_2011,predictions_2001,predictions_2003,predictions_2005,predictions_2007,predictions_2009,predictions_2011
0,1,2,2,1,2.0,3.0,2.0,2.0,2.0,2.0
1,2,2,2,1,3.0,2.0,2.0,2.0,2.0,2.0
3,4,3,3,2,2.0,3.0,3.0,3.0,3.0,3.0
4,5,2,2,1,3.0,3.0,3.0,3.0,2.0,2.0
5,6,2,2,1,2.0,3.0,2.0,2.0,2.0,2.0


In [387]:
# feature_cols = [indicator+'_2011','LIT_2001','FEMP_2001','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

# For Lit
# feature_cols = [indicator+'_2011','FEMP_2011','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

# For FEMP
feature_cols = [indicator+'_2011','LIT_2011','predictions_2003','predictions_2005', 'predictions_2007', 'predictions_2009']

target = indicator+'_2001'

In [388]:
X = data[feature_cols].values
y = data[target].values

In [389]:
val_f1score = []
val_accscore = []
train_f1score = []
train_accscore = []
# f1score = []
# accscore = []
counter=0
for train_index, test_index in cv.split(X):
    
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    sm = SMOTE(random_state=42)
    X_train, y_train = sm.fit_resample(X_train, y_train)
    
    best_xgb = xgbc.fit(X_train, y_train)
    
#     prediction_2011 = best_xgb.predict(X_2011)
#     f1_weight = f1_score(y_2011, prediction_2011, average='weighted')
#     acc = accuracy_score(y_2011, prediction_2011)

#     f1score.append(f1_weight)
#     accscore.append(acc)

    predictions = best_xgb.predict(X_test)
    predictions_train = best_xgb.predict(X_train)

    f1_weight = f1_score(y_test, predictions, average='weighted')
    f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
    acc = accuracy_score(y_test, predictions)
    acc_train = accuracy_score(y_train, predictions_train)
    print(counter, f1_weight, f1_weight_train, acc, acc_train)
    counter=counter+1

    val_f1score.append(f1_weight)
    train_f1score.append(f1_weight_train)
    val_accscore.append(acc)
    train_accscore.append(acc_train)

val_f1score = np.array(val_f1score).mean()
train_f1score = np.array(train_f1score).mean()
val_accscore = np.array(val_accscore).mean()
train_accscore = np.array(train_accscore).mean()
print('------')
print(val_f1score, train_f1score, val_accscore, train_accscore)

0 0.9051435122446227 0.9248999569764819 0.9051724137931034 0.9246704331450094
1 0.8707362188380081 0.9332676739059218 0.8706896551724138 0.9333333333333333
2 0.8519091593804237 0.9317632724252536 0.853448275862069 0.9314285714285714
3 0.8649904829356824 0.9366109001258256 0.8620689655172413 0.9365079365079365
4 0.8791635528984734 0.9295688030233998 0.8793103448275862 0.9296296296296296
------
0.8743885852594421 0.9312221212913764 0.8741379310344828 0.931113980808896


In [390]:
file = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/input_data_2011-19/FEMP_CC.csv")

In [346]:
# FOR ALL Except LIT and FEMP
ground_truth = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/District - Ground Truth - 2011&2001.csv")
gt_cols = ['census_code','MSW_2011']

ground_truth = ground_truth[gt_cols]
femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")
cols_2 = ['census_code', 'LIT_2011', 'FEMP_2011']
femp_lit = femp_lit[cols_2]

In [347]:
print(ground_truth.shape)
print(femp_lit.shape)
gt_femlit = ground_truth.merge(femp_lit, on='census_code', how='left')
gt_femlit.dropna(inplace=True)
print(gt_femlit.shape)

(593, 2)
(593, 3)
(593, 4)


In [391]:
# For LIT and FEMP
femp_lit = pd.read_csv("/Users/arpitjain/Downloads/SatPRo/2001_L7_data/ChangeClassifier/FEMP&LIT.csv")
cols_2 = ['census_code', 'LIT_2011', 'FEMP_2011']
gt_femlit = femp_lit[cols_2]

In [392]:
X_test_fut = gt_femlit.merge(file, on='census_code', how='left')
X_test_fut.dropna(inplace=True)
X_test_fut.shape

(589, 8)

In [393]:
X_test_fut.head()

,census_code,LIT_2011,FEMP_2011,predictions_2011,predictions_2013,predictions_2015,predictions_2017,predictions_2019
0,1,1,2,2.0,2.0,2.0,2.0,3.0
1,2,1,2,2.0,2.0,2.0,2.0,2.0
2,3,2,3,3.0,2.0,3.0,3.0,3.0
3,4,2,3,3.0,2.0,2.0,3.0,2.0
4,5,1,2,2.0,3.0,3.0,3.0,3.0


In [394]:
X_test_fut.head()

,census_code,LIT_2011,FEMP_2011,predictions_2011,predictions_2013,predictions_2015,predictions_2017,predictions_2019
0,1,1,2,2.0,2.0,2.0,2.0,3.0
1,2,1,2,2.0,2.0,2.0,2.0,2.0
2,3,2,3,3.0,2.0,3.0,3.0,3.0
3,4,2,3,3.0,2.0,2.0,3.0,2.0
4,5,1,2,2.0,3.0,3.0,3.0,3.0


In [395]:
X_test_fut.columns

Index(['census_code', 'LIT_2011', 'FEMP_2011', 'predictions_2011',
       'predictions_2013', 'predictions_2015', 'predictions_2017',
       'predictions_2019'],
      dtype='object')

In [396]:
census_codes = X_test_fut['census_code'].values

In [397]:
# NOTE that order should be same

# cols_fut = ['MSW_2011', 'LIT_2011', 'FEMP_2011','predictions_2011', 'predictions_2013', 'predictions_2015','predictions_2017']

# For Lit
cols_fut = ['LIT_2011', 'FEMP_2011','predictions_2011', 'predictions_2013', 'predictions_2015','predictions_2017']

# For FEMP
cols_fut = ['FEMP_2011','LIT_2011','predictions_2011', 'predictions_2013', 'predictions_2015','predictions_2017']

X_test_fut = X_test_fut[cols_fut]

In [398]:
X_test_fut = X_test_fut.values

X_test_fut.shape

X_train.shape

(540, 6)

In [399]:
predictions_future = best_xgb.predict(X_test_fut)

predictions_future.shape

census_codes.shape

final_predictions = np.stack((census_codes, predictions_future))

final_predictions.shape

final_predictions_df = pd.DataFrame(final_predictions)

final_predictions_df = final_predictions_df.transpose()

final_predictions_df.columns = ['census_code', 'prediction_2019']

In [400]:
final_predictions_df.head()

,census_code,prediction_2019
0,1,2
1,2,2
2,3,3
3,4,3
4,5,2


In [401]:
final_predictions_df['prediction_2019'].value_counts()

2    220
3    194
1    175
Name: prediction_2019, dtype: int64

In [402]:
final_predictions_df

,census_code,prediction_2019
0,1,2
1,2,2
2,3,3
3,4,3
4,5,2
...,...,...
584,635,3
585,636,3
586,637,3
587,638,3


In [403]:
final_predictions_df.to_csv('Future_FEMP_2019_UsingM2011.csv', index=False)